In [100]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [101]:
df = pd.read_csv('../elections_33.csv', encoding='utf-8')

In [102]:
df.head()

,Unnamed: 0,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,% Voix/Exp,Candidat_ID,Année,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Blancs,N°Panneau
0,49596,33,GIRONDE,1,ABZAC,1091,195,17.87,896,82.13,...,16.63,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49597,33,GIRONDE,2,AILLAS,530,92,17.36,438,82.64,...,8.49,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49598,33,GIRONDE,3,AMBARES & LAGRAVE,6930,1306,18.85,5624,81.15,...,16.40,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49599,33,GIRONDE,4,AMBES,1834,295,16.09,1539,83.91,...,11.89,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49600,33,GIRONDE,5,ANDERNOS LES BAINS,6551,1422,21.71,5129,78.29,...,13.44,1,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Création d'un nouveau dataframe propre

In [103]:
# create new dataframe with only the columns we need
cols = [
    'Annee', 'Code de la commune', 'Libellé de la commune', 'Inscrits', 'Abstentions', 'Votants', 'Exprimés',
    'Blancs_et_nuls',
    'Sexe', 'Nom Complet', 'Voix', 'Candidat_ID',
]

In [104]:
df_final = pd.DataFrame(columns=cols)

In [105]:
df_final['Annee'] = df['Année']
df_final['Code de la commune'] = df['Code de la commune']
df_final['Libellé de la commune'] = df['Libellé de la commune']
df_final['Inscrits'] = df['Inscrits']
df_final['Abstentions'] = df['Abstentions']
df_final['Votants'] = df['Votants']
df_final['Exprimés'] = df['Exprimés']

# Blancs_et_nuls we need to take the 'Blancs_et_nuls' column and add 'Blancs' and 'Nuls' columns on top of it
df['Blancs'] = df['Blancs'].fillna(0)
df['Nuls'] = df['Nuls'].fillna(0)
df_final['Blancs_et_nuls'] = df['Blancs_et_nuls'] + df['Blancs'] + df['Nuls']

df_final['Sexe'] = df['Sexe'].fillna('N/A')
df_final['Nom Complet'] = df['Nom'] + ' ' + df["Prénom"]
df_final['Voix'] = df['Voix'].fillna(0)
df_final['Candidat_ID'] = df['Candidat_ID'].fillna(0)

In [106]:
political_lean = {
    'LE PEN J.MARIE': -0.6,  # Far-right
    'CHIRAC JACQUES': 0.4,  # Center-right
    'JOSPIN LIONEL': -0.4,  # Center-left
    'VOYNET DOMINIQUE': -0.8,  # Green left
    'LEPAGE CORINNE': -0.6,  # Ecologist
    'LE PEN JEAN-MARIE': -0.8,  # Far-right
    'BOUTIN CHRISTINE': 0.6,  # Christian democracy
    'MADELIN ALAIN': 0.6,  # Liberal conservatism
    'LAGUILLER ARLETTE': -1.0,  # Far-left
    'BESANCENOT OLIVIER': -1.0,  # Far-left
    'BUFFET MARIE-GEORGE': -0.6,  # Left-wing
    'SCHIVARDI GÉRARD': 0.0,  # Centrist
    'BAYROU FRANÇOIS': 0.2,  # Center to center-right
    'BOVÉ JOSÉ': -0.6,  # Ecologist
    'DE VILLIERS PHILIPPE': 0.8,  # Right-wing
    'ROYAL SÉGOLÈNE': -0.4,  # Center-left
    'NIHOUS FRÉDÉRIC': 0.0,  # Centrist
    'LAGUILLER ARLETTE': -1.0,  # Far-left
    'SARKOZY NICOLAS': 0.6,  # Center-right
    'CHEMINADE JACQUES': -0.2,  # Center-left
    'LE PEN MARINE': -0.8,  # Far-right
    'MACRON EMMANUEL': 0.2,  # Center to center-right
    'MÉLENCHON JEAN-LUC': -0.8,  # Far-left
    'FILLON FRANÇOIS': 0.6,  # Center-right
    'HAMON BENOÎT': -0.4,  # Center-left
    'LASSALLE JEAN': 0.0,  # Centrist
    'DUPONT-AIGNAN NICOLAS': 0.4,  # Center-right
    'POUTOU PHILIPPE': -1.0,  # Far-left
    'ASSELINEAU FRANÇOIS': 0.0,  # Centrist
    'ARTHAUD NATHALIE': -1.0,  # Far-left
    'HIDALGO ANNE': -0.4,  # Center-left
}

df_final['Orientation'] = df_final['Nom Complet'].map(political_lean)

In [107]:
df_final.head()

,Annee,Code de la commune,Libellé de la commune,Inscrits,Abstentions,Votants,Exprimés,Blancs_et_nuls,Sexe,Nom Complet,Voix,Candidat_ID,Orientation
0,1995,1,ABZAC,1091,195,896,860,36.0,M,LE PEN J.MARIE,143,1,-0.6
1,1995,2,AILLAS,530,92,438,424,14.0,M,LE PEN J.MARIE,36,1,-0.6
2,1995,3,AMBARES & LAGRAVE,6930,1306,5624,5468,156.0,M,LE PEN J.MARIE,897,1,-0.6
3,1995,4,AMBES,1834,295,1539,1506,33.0,M,LE PEN J.MARIE,179,1,-0.6
4,1995,5,ANDERNOS LES BAINS,6551,1422,5129,4994,135.0,M,LE PEN J.MARIE,671,1,-0.6


## Calcul des pourcentages

In [108]:
df_final['Blancs_et_nuls'] = df_final['Blancs_et_nuls'].fillna(0)

In [109]:
df_final['Pourcentage_Blancs_et_nuls'] = df_final['Blancs_et_nuls'] / df_final['Exprimés'] * 100
df_final['Pourcentage_Abstentions'] = df_final['Abstentions'] / df_final['Inscrits'] * 100
df_final['Pourcentage_Votants'] = df_final['Votants'] / df_final['Inscrits'] * 100

In [110]:
# export to csv
df_final.head()

,Annee,Code de la commune,Libellé de la commune,Inscrits,Abstentions,Votants,Exprimés,Blancs_et_nuls,Sexe,Nom Complet,Voix,Candidat_ID,Orientation,Pourcentage_Blancs_et_nuls,Pourcentage_Abstentions,Pourcentage_Votants
0,1995,1,ABZAC,1091,195,896,860,36.0,M,LE PEN J.MARIE,143,1,-0.6,4.186047,17.873511,82.126489
1,1995,2,AILLAS,530,92,438,424,14.0,M,LE PEN J.MARIE,36,1,-0.6,3.301887,17.358491,82.641509
2,1995,3,AMBARES & LAGRAVE,6930,1306,5624,5468,156.0,M,LE PEN J.MARIE,897,1,-0.6,2.852963,18.845599,81.154401
3,1995,4,AMBES,1834,295,1539,1506,33.0,M,LE PEN J.MARIE,179,1,-0.6,2.191235,16.085060,83.914940
4,1995,5,ANDERNOS LES BAINS,6551,1422,5129,4994,135.0,M,LE PEN J.MARIE,671,1,-0.6,2.703244,21.706610,78.293390


In [111]:
df_final.to_csv('elections_33_clean.csv', index=False)
for col in df_final.columns:
    print(col, df_final[col].dtype)

Annee int64
Code de la commune int64
Libellé de la commune object
Inscrits int64
Abstentions int64
Votants int64
Exprimés int64
Blancs_et_nuls float64
Sexe object
Nom Complet object
Voix int64
Candidat_ID int64
Orientation float64
Pourcentage_Blancs_et_nuls float64
Pourcentage_Abstentions float64
Pourcentage_Votants float64


# Ajout inflation

In [112]:
df_inflation = pd.read_csv('./clean_inflation.csv', encoding='utf-8')

In [113]:
df_inflation.head()

,Période,Indice des prix
0,2024-03,2.3
1,2024-02,3.6
2,2024-01,4.5
3,2023-12,5.9
4,2023-11,7.0


In [114]:
# Convert the "Période" column to datetime format
df_inflation['Période'] = pd.to_datetime(df_inflation['Période'], format='%Y-%m')

# Extract the year from the "Période" column
df_inflation['Year'] = df_inflation['Période'].dt.year

# Group the data by 'Year' and calculate the mean of "Indice des prix"
mean_indice_by_year = df_inflation.groupby('Year')['Indice des prix'].mean().reset_index()


In [115]:
df_inflation["Year"].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012], dtype=int32)

In [118]:
# Fill the 'Inflation' column with the mean inflation of the corresponding year
df_final['Inflation'] = None

# If not available, do nothing
for year in df_inflation["Year"].unique():
    inflation = mean_indice_by_year[mean_indice_by_year['Year'] == year]['Indice des prix'].values[0]
    df_final.loc[df_final['Annee'] == year, 'Inflation'] = inflation

In [119]:
df_final

,Annee,Code de la commune,Libellé de la commune,Inscrits,Abstentions,Votants,Exprimés,Blancs_et_nuls,Sexe,Nom Complet,Voix,Candidat_ID,Orientation,Pourcentage_Blancs_et_nuls,Pourcentage_Abstentions,Pourcentage_Votants,Inflation
0,1995,1,ABZAC,1091,195,896,860,36.0,M,LE PEN J.MARIE,143,1,-0.6,4.186047,17.873511,82.126489,None
1,1995,2,AILLAS,530,92,438,424,14.0,M,LE PEN J.MARIE,36,1,-0.6,3.301887,17.358491,82.641509,None
2,1995,3,AMBARES & LAGRAVE,6930,1306,5624,5468,156.0,M,LE PEN J.MARIE,897,1,-0.6,2.852963,18.845599,81.154401,None
3,1995,4,AMBES,1834,295,1539,1506,33.0,M,LE PEN J.MARIE,179,1,-0.6,2.191235,16.085060,83.914940,None
4,1995,5,ANDERNOS LES BAINS,6551,1422,5129,4994,135.0,M,LE PEN J.MARIE,671,1,-0.6,2.703244,21.706610,78.293390,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21671,2022,554,Yvrac,1138,197,941,924,0.0,M,POUTOU Philippe,7,10,NaN,0.000000,17.311072,82.688928,6.1
21672,2022,555,Marcheprime,1226,250,976,953,0.0,M,POUTOU Philippe,9,10,NaN,0.000000,20.391517,79.608483,6.1
21673,2022,555,Marcheprime,1543,292,1251,1213,0.0,M,POUTOU Philippe,14,10,NaN,0.000000,18.924174,81.075826,6.1
21674,2022,555,Marcheprime,895,162,733,719,0.0,M,POUTOU Philippe,13,10,NaN,0.000000,18.100559,81.899441,6.1
